In [1]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, inspect, text
from sqlalchemy import Column, Integer, String, Boolean, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base
from werkzeug.security import generate_password_hash

In [2]:
# Формируем строку подключения
connection_string = open('../interface_db.txt', 'r').read()
# Создаем движок подключения
engine = create_engine(connection_string)
# Проверяем подключение
try:
    with engine.connect() as connection:
        print("Подключение к базе данных успешно!")
except Exception as e:
    print(f"Ошибка подключения к базе данных: {e}")
finally:
    connection.close()

Подключение к базе данных успешно!


In [3]:
inspector = inspect(engine)
tables = inspector.get_table_names()
display(pd.DataFrame(tables, columns=['Table Name']))

,Table Name
0,model_meta
1,model_method
2,model_hyperparam
3,interface_user


# Пользователь

## Создание таблицы user

In [29]:
Base = declarative_base()

In [14]:
class InterfaceUser(Base):
    __tablename__ = 'interface_user'

    id = Column(Integer, primary_key=True)
    first_name = Column(String, nullable=False)  # Имя
    last_name = Column(String, nullable=False)   # Фамилия
    username = Column(String, unique=True, nullable=False)  # Логин
    password = Column(String, nullable=False)  # Пароль

In [10]:
Base.metadata.create_all(engine)

In [6]:
#Base.metadata.drop_all(engine, tables=[InterfaceUser.__table__])  # удаление таблицы

## Добавление пользователей

In [31]:
Session = sessionmaker(bind=engine)
session = Session()

In [32]:
def add_user(first_name, last_name, username, password):
    user = InterfaceUser(
        first_name=first_name,
        last_name=last_name,
        username=username,
        password=generate_password_hash(password)
    )
    session.add(user)
    session.commit()
    print(f"Пользователь {username} добавлен!")

## Получение всех пользователей

In [11]:
query = text("SELECT * FROM interface_user")
with engine.connect() as connection:
    result = connection.execute(query)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())

# Выведите DataFrame
print(df.head())

   id first_name  last_name   username  \
0   1    Дмитрий  Кондрашов  dmitrykon   
1   2      Артём   Петриков  APetrikov   

                                            password  
0  scrypt:32768:8:1$vvIpmHn0o6OCKPpC$b22cf5e24227...  
1  scrypt:32768:8:1$xneRAPf0w7HSALty$9ec963de89fc...  


# Модели

## Создание таблицы model_method

In [5]:
Base = declarative_base()

In [15]:
class ModelMethod(Base):
    __tablename__ = 'model_method'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)  # Название метода моделирования

In [7]:
Base.metadata.create_all(engine)

### Добавление методов моделирования

In [7]:
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
def add_model_method(name):
    model_method = ModelMethod(
        name=name,
    )
    session.add(model_method)
    session.commit()
    print(f"Метод моделирования {name} добавлен!")

In [9]:
add_model_method("CatBoostRegressor")

Метод моделирования CatBoostRegressor добавлен!


### Получение всех методов

In [10]:
query = text("SELECT * FROM model_method")
with engine.connect() as connection:
    result = connection.execute(query)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())

print(df.head())

   id               name
0   1  CatBoostRegressor


## Создание таблицы model_meta

In [14]:
Base = declarative_base()

In [15]:
class ModelMeta(Base):
    __tablename__ = 'model_meta'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)  # Название модели
    author = Column(Integer, ForeignKey("interface_user.id"), nullable=False)  # Автор модели
    description = Column(String, nullable=True)  # Описание
    creation_on = Column(DateTime, default=datetime.now, nullable=False)  # Время создания
    last_changed = Column(Integer, ForeignKey("interface_user.id"), nullable=False)  # Пользователь, изменивший
    # модель последний раз
    updated_on = Column(DateTime, default=datetime.now, onupdate=datetime.now, nullable=False)  # Время, когда
    # последний раз модель была изменена пользователем
    finalized = Column(Boolean, default=False, nullable=False)  # Модель финализирована?
    state = Column(Integer, default=0, nullable=False)  # Состояние модели:
    # 0 - создана запись о модели в БД (автор/дата/описание/типовая позиция)
    # 1 - выбран метод моделирования
    # 2 - выбраны гиперпараметры метода моделирования
    # 3 - модель обучена -> changed=1
    # 4 - скорректированы отрицательные значения -> finalized=1
    # 5 - модель финализирована
    # 6 - модель используется -> used=1
    profession = Column(Integer, nullable=True)  # Номер типовой профессии
    method = Column(Integer, ForeignKey("model_method.id"), nullable=True)  # Метод моделирования
    used = Column(Boolean, default=False, nullable=False)  # Используется в калькуляторе?
    retrained = Column(Integer, default=0, nullable=False)  # Сколько раз была переобучена
    orig = Column(Integer, ForeignKey("model_meta.id"), nullable=True)  # Ссылка на оригинал модели, если копия

In [17]:
Base.metadata.create_all(engine)

### Вывод всех записей

In [8]:
try:
    with engine.connect() as connection:
        query = text("SELECT * FROM model_meta")
        result = connection.execute(query)
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        display(df.head())
except Exception as e:
    print(f"Ошибка при выполнении операции: {e}")

,id,name,author,description,creation_on,last_changed,updated_on,finalized,state,profession,method,used,retrained,orig


In [7]:
# clear_table_query = text("DELETE FROM model_meta CASCADE")
#
# try:
#     with engine.connect() as connection:
#         # Выполнение запроса на очистку таблицы
#         connection.execute(clear_table_query)
#         print("Содержимое таблицы 'model_meta' успешно очищено.")
#
#         # Проверка содержимого таблицы после очистки
#         query = text("SELECT * FROM model_meta")
#         result = connection.execute(query)
#         df = pd.DataFrame(result.fetchall(), columns=result.keys())
#
#         print(df.head())
#
#         connection.commit()
# except Exception as e:
#     print(f"Ошибка при выполнении операции: {e}")

Содержимое таблицы 'model_meta' успешно очищено.
Empty DataFrame
Columns: [id, name, author, description, creation_on, last_changed, updated_on, finalized, state, profession, method, used, retrained, orig]
Index: []


## Создание таблицы model_hyperparam

In [ ]:
Base = declarative_base()

In [19]:
class ModelHyperparam(Base):
    __tablename__ = 'model_hyperparam'

    id = Column(Integer, primary_key=True)
    model_id = Column(Integer, ForeignKey("model_meta.id"), nullable=False)
    name = Column(String, nullable=False)  # Название гиперпараметра модели
    value = Column(String, nullable=False)  # Значение гиперпараметра модели

In [20]:
Base.metadata.create_all(engine)